# General Settings

In [1]:
import os
import json
import pickle
from datetime import datetime
from pathlib import Path
from typing import List, Tuple, Dict, Any, Optional

import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import gc

import statsmodels.api as sm
from statsmodels.discrete.discrete_model import NegativeBinomial
from statsmodels.discrete.discrete_model import Poisson as PoisDM
from statsmodels.tools.sm_exceptions import ConvergenceWarning, HessianInversionWarning
from sklearn.linear_model import PoissonRegressor

In [ ]:
# Configuration
FUTURE_SRC_DIR = Path("processed_tmp/future") # Directory where training data is stored
ENGINE = "fastparquet" # Reading method
DATE_COL = "sold_date"
BAN_COLS_COMMON = {DATE_COL}
BAN_PREFIXES = []

TARGET_BASE = "future_sales"
H_TARGET_FMT = TARGET_BASE + "_{}"
SUM_TARGET = "future_sales_sum"

K_CANDIDATES = list(range(60, 301, 60)) # List of numbers of features to select
NB_TOP = 60 # Number of features to use for NB or Poisson model regression

LGB_NUM_BOOST_ROUND = 8000 # Maximum number of LightGBM training epochs
LGB_EARLY_STOP_ROUNDS = 200 # Number of epochs for LightGBM Early Stopping
RANDOM_SEED = 1234

# Set categorical variables
CAT_INT_COLS = ["jan_code", "store_name", "中分類名", "price_category"]

# Settings for outlier handling (upper and lower bounds)
WINSOR_LO_Q = 0.5 / 100
WINSOR_HI_Q = 99.5 / 100
# Setting for the position of the value used as the reference for arcsinh scaling
ASINH_Q = 0.90

# --- Output directory settings ---
RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")
ROOT_OUT_DIR = Path(f"lgb_nb_stacking_kgrid_all_{RUN_TAG}")
ROOT_OUT_DIR.mkdir(parents=True, exist_ok=True)

EVAL_OUT = ROOT_OUT_DIR / "eval_results"
EVAL_OUT.mkdir(parents=True, exist_ok=True)
EVAL_CSV_PATH = EVAL_OUT / "metrics_all_tasks.csv"
EVAL_JSON_PATH = EVAL_OUT / "metrics_summary.json"

print(f"Output directory created: {ROOT_OUT_DIR.resolve()}")

In [ ]:
def save_pickle(obj: Any, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "wb") as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_src_df(task: str) -> Tuple[pd.DataFrame, str]:
    """Load task-specific data (exclude only target variable NaN, keep others)"""
    if task == "sum":
        src_path = FUTURE_SRC_DIR / "future_sum26.parquet"
        target = SUM_TARGET
    else:
        N = int(task[1:])
        src_path = FUTURE_SRC_DIR / f"future_h{N}.parquet"
        target = H_TARGET_FMT.format(N)

    print(f"      Loading data: {src_path}")
    if not src_path.exists():
        raise FileNotFoundError(f"{src_path} not found.")

    df = pd.read_parquet(src_path, engine=ENGINE)
    print(f"      - Completed: {len(df):,} rows × {len(df.columns)} columns")
    
    if DATE_COL not in df.columns:
        raise KeyError(f"{DATE_COL} not found.")
    df[DATE_COL] = pd.to_datetime(df[DATE_COL])

    if target not in df.columns:
        raise KeyError(f"Target variable {target} not found.")
    print(f"      - Target variable: {target}")
    
    # Exclude only NaN in target column (keep other missing values)
    before_len = len(df)
    df = df.dropna(subset=[target]).copy()
    after_len = len(df)
    if before_len != after_len:
        print(f"      - Target variable NaN removed: {before_len:,} → {after_len:,} rows")

    # Clip target variable to non-negative integers
    df[target] = np.maximum(pd.to_numeric(df[target], errors="coerce").fillna(0).astype("Int64"), 0).astype(np.int64)
    
    # Sort by time series
    df = df.sort_values(DATE_COL).reset_index(drop=True)
    print(f"      - Time series sorting completed: {df[DATE_COL].min()} ～ {df[DATE_COL].max()}")
    return df, target

def time_split_by_date(df: pd.DataFrame, train_ratio: float = 0.8, date_col: str = DATE_COL) -> Tuple[pd.Timestamp, np.ndarray, np.ndarray]:
    """Time series 8:2 split (strict split without crossing same dates)"""
    if len(df) == 0:
        return pd.NaT, np.array([], dtype=int), np.array([], dtype=int)

    # Assume already sorted by time series
    df_sorted = df.sort_values(date_col)
    n = len(df_sorted)
    
    # Determine 80% position based on rows
    cut_row = max(1, int(n * train_ratio)) - 1  # 0-index
    raw_cutoff = df_sorted.iloc[cut_row][date_col]
    cutoff_date = pd.to_datetime(raw_cutoff)
    
    print(f"       - Cutoff date determined: {cutoff_date} (80% position based on row count)")

    # Split using <= / > to avoid crossing same dates
    train_mask = df[date_col] <= cutoff_date
    val_mask = df[date_col] > cutoff_date
    
    print(f"        - Initial split: train={train_mask.sum():,} rows, val={val_mask.sum():,} rows")

    # Handle case when val is empty
    if val_mask.sum() == 0:
        print(f"        - val is empty, lowering to previous unique date")
        uniq_dates = np.sort(df[date_col].unique())
        pos = np.searchsorted(uniq_dates, cutoff_date, side="left")
        if pos > 0:
            cutoff_date = uniq_dates[pos - 1]
            train_mask = df[date_col] <= cutoff_date
            val_mask = df[date_col] > cutoff_date
            print(f"        - After lowering: train={train_mask.sum():,} rows, val={val_mask.sum():,} rows")
        
        # If val is still empty, set last date to val
        if val_mask.sum() == 0:
            print(f"        - Setting last date to val")
            last_date = uniq_dates[-1]
            train_mask = df[date_col] < last_date
            val_mask = df[date_col] >= last_date
            cutoff_date = last_date
            print(f"        - Final split: train={train_mask.sum():,} rows, val={val_mask.sum():,} rows")

    tr_idx = df.index[train_mask].to_numpy()
    va_idx = df.index[val_mask].to_numpy()
    
    # Verify that same dates do not cross train/val
    train_dates = set(df.loc[train_mask, date_col].unique())
    val_dates = set(df.loc[val_mask, date_col].unique())
    overlap = train_dates & val_dates
    if overlap:
        raise ValueError(f"Same dates cross train/val: {overlap}")
    
    print(f"        - Split completed: cutoff_date={cutoff_date}, train={len(tr_idx):,} rows, val={len(va_idx):,} rows")
    return cutoff_date, tr_idx, va_idx

def pick_feature_cols(df: pd.DataFrame, target_col: str) -> Tuple[List[str], List[str]]:
    """Extract feature columns"""
    cols = []
    ban_cols = set(BAN_COLS_COMMON) | {target_col}
    
    for c in df.columns:
        if c in ban_cols:
            continue
        # Exclude prefixes that may cause future leakage
        if any(c.startswith(p) for p in BAN_PREFIXES):
            continue
        # Exclude datetime columns
        if np.issubdtype(df[c].dtype, np.datetime64):
            continue
        cols.append(c)
    
    # Numeric columns (excluding categorical integer columns)
    num_cols = [c for c in cols if pd.api.types.is_numeric_dtype(df[c]) and c not in CAT_INT_COLS]
    # Categorical columns (only predefined categorical integer columns)
    cat_cols = [c for c in CAT_INT_COLS if c in df.columns]
    
    print(f"        - Feature selection: numeric={len(num_cols)}, categorical={len(cat_cols)}, excluded={len(df.columns) - len(num_cols) - len(cat_cols) - len(ban_cols)}")
    return num_cols, cat_cols

def make_3fold_80_20_indices(df: pd.DataFrame, date_col: str = DATE_COL) -> List[Tuple[np.ndarray, np.ndarray]]:
    """3-fold Time Series Split (divide into 3 equal parts based on sold_date, 8:2 split within each fold)"""
    # Get and sort all sold_date data
    df_sorted = df.sort_values(date_col).reset_index(drop=True)
    unique_dates = df_sorted[date_col].unique()
    n_dates = len(unique_dates)
    
    print(f"        - Number of unique dates: {n_dates:,} days")
    
    # Determine dates for 3-way division
    date_1_3 = unique_dates[n_dates // 3]
    date_2_3 = unique_dates[2 * n_dates // 3]
    
    print(f"        - 3-way division dates: {date_1_3}, {date_2_3}")
    
    folds = []
    
    for k in range(3):
        # Determine date range for each fold
        if k == 0:
            # Fold 1: First 1/3
            fold_start_date = unique_dates[0]
            fold_end_date = date_1_3
        elif k == 1:
            # Fold 2: Middle 1/3
            fold_start_date = date_1_3
            fold_end_date = date_2_3
        else:
            # Fold 3: Last 1/3
            fold_start_date = date_2_3
            fold_end_date = unique_dates[-1]
        
        # Get data within each fold
        fold_mask = (df_sorted[date_col] >= fold_start_date) & (df_sorted[date_col] <= fold_end_date)
        fold_df = df_sorted[fold_mask].reset_index(drop=True)
        fold_dates = fold_df[date_col].unique()
        n_fold_dates = len(fold_dates)
        
        # Determine cutoff date for 8:2 split within fold
        train_end_date_idx = int(n_fold_dates * 0.8)
        train_end_date = fold_dates[train_end_date_idx]
        
        # Get train/val indices
        train_mask = fold_df[date_col] <= train_end_date
        val_mask = fold_df[date_col] > train_end_date
        
        # Convert to original DataFrame indices
        train_idx = fold_df[train_mask].index.to_numpy()
        val_idx = fold_df[val_mask].index.to_numpy()
        
        print(f"        - Fold {k+1}: Date range [{fold_start_date}～{fold_end_date}]")
        print(f"          - train: {len(train_idx):,} rows (～{train_end_date}), val: {len(val_idx):,} rows")
        
        folds.append((train_idx, val_idx))
    
    return folds

# General Functions for LightGBM Training

In [ ]:
def lgb_preprocess_for_lgb(df: pd.DataFrame, num_cols: List[str], cat_cols: List[str]) -> pd.DataFrame:
    X = df[num_cols + cat_cols].copy()
    for c in num_cols:
        X[c] = X[c].astype(float).fillna(0.0)
    for c in cat_cols:
        X[c] = X[c].astype("Int64").fillna(-1)
    return X

def lgb_train_eval(
    X_tr: pd.DataFrame, y_tr: np.ndarray,
    X_va: Optional[pd.DataFrame], y_va: Optional[np.ndarray],
    cat_cols: List[str],
    num_boost_round: int = LGB_NUM_BOOST_ROUND,
    early_stopping_rounds: int = LGB_EARLY_STOP_ROUNDS
) -> Tuple[lgb.Booster, np.ndarray, float]:
    dtrain = lgb.Dataset(X_tr, label=y_tr, categorical_feature=cat_cols, free_raw_data=True)
    valid_sets = [dtrain]; valid_names = ["train"]
    callbacks = []
    dvalid = None
    if X_va is not None and y_va is not None:
        dvalid = lgb.Dataset(X_va, label=y_va, categorical_feature=cat_cols, reference=dtrain, free_raw_data=True)
        valid_sets = [dtrain, dvalid]; valid_names = ["train", "valid"]
        callbacks = [lgb.early_stopping(early_stopping_rounds, verbose=False)]

    params = dict(
        objective="tweedie", tweedie_variance_power=1.5,
        learning_rate=0.05, num_leaves=64, min_data_in_leaf=100,
        feature_fraction=0.9, bagging_fraction=0.9, bagging_freq=1,
        lambda_l1=0.0, max_depth=-1, seed=RANDOM_SEED, verbose=-1,
        metric="rmse", first_metric_only=True, feature_pre_filter=False,
    )

    booster = lgb.train(
        params=params,
        train_set=dtrain,
        num_boost_round=num_boost_round,
        valid_sets=valid_sets,
        valid_names=valid_names,
        callbacks=callbacks
    )

    del dtrain
    if dvalid is not None:
        del dvalid
    gc.collect()

    if X_va is None or y_va is None:
        return booster, np.array([]), np.nan

    pred_va = booster.predict(X_va, num_iteration=booster.best_iteration)
    rmse = float(np.sqrt(np.mean((y_va - pred_va) ** 2)))
    return booster, pred_va, rmse

# LightGBM Steps 1 & 2 Training

- Step 1 : Train using all features with data split into 3 folds to create **feature importance rankings**
- Step 2 : Evaluate accuracy when training with **Top K** features using the feature importance rankings, and select the optimal number of features

In [ ]:
def lgb_fold_importances_raw(
    df_train: pd.DataFrame, target_col: str,
    num_cols: List[str], cat_cols: List[str],
    folds: List[Tuple[np.ndarray, np.ndarray]],
    out_dir: Path
) -> pd.DataFrame:
    """Calculate feature importance using only train data (prevent time series leakage)"""
    print(f"    Starting LightGBM feature importance calculation (train only, {len(folds)} folds)")
    os.makedirs(out_dir, exist_ok=True)
    all_imps = []
    cols = num_cols + cat_cols
    y_train = df_train[target_col].to_numpy()
    print(f"    - Total number of features: {len(cols)} (numeric:{len(num_cols)}, categorical:{len(cat_cols)})")
    print(f"    - Train data: {len(df_train):,} rows")

    for i, (tr_idx, va_idx) in enumerate(folds, start=1):
        print(f"      Processing Fold {i}/{len(folds)}...")
        fdir = out_dir / f"fold{i}"
        fdir.mkdir(parents=True, exist_ok=True)

        # Fold split within train data
        X_tr = lgb_preprocess_for_lgb(df_train.iloc[tr_idx], num_cols, cat_cols)
        X_va = lgb_preprocess_for_lgb(df_train.iloc[va_idx], num_cols, cat_cols)
        y_tr = y_train[tr_idx]; y_va = y_train[va_idx]
        print(f"        - Training data: {len(X_tr):,} rows")
        print(f"        - Validation data: {len(X_va):,} rows")

        booster, pred_va, rmse = lgb_train_eval(X_tr, y_tr, X_va, y_va, cat_cols)
        save_pickle(booster, fdir / "lgb_importance_model.pkl")
        
        # Calculate R²
        ss_res = np.sum((y_va - pred_va) ** 2)
        ss_tot = np.sum((y_va - np.mean(y_va)) ** 2)
        r2 = 1.0 - (ss_res / (ss_tot + 1e-9))
        print(f"        - Training completed: RMSE={rmse:.6f}, R²={r2:.6f}")

        imp = pd.DataFrame({
            "feature": cols,
            "gain": booster.feature_importance(importance_type="gain"),
            "split": booster.feature_importance(importance_type="split"),
            "fold": i,
            "rmse_val": rmse
        })
        imp.sort_values(["gain", "split"], ascending=False).to_csv(fdir / "feature_importance.csv", index=False)
        all_imps.append(imp)

        del booster, X_tr, X_va, y_tr, y_va, imp
        gc.collect()

    print(f"    Aggregating importance...")
    imp_cat = pd.concat(all_imps, axis=0, ignore_index=True)
    avg = (imp_cat.groupby("feature", as_index=False)
                 .agg(gain_mean=("gain","mean"), split_mean=("split","mean")))
    avg = avg.sort_values(["gain_mean", "split_mean"], ascending=False).reset_index(drop=True)
    avg.to_csv(out_dir / "feature_importance_avg.csv", index=False)
    print(f"    - Importance aggregation completed: {len(avg)} features")

    del all_imps, imp_cat
    gc.collect()
    return avg

def lgb_kgrid_cv_raw(
    df_train: pd.DataFrame, target_col: str,
    num_cols: List[str], cat_cols: List[str],
    avg_importance: pd.DataFrame,
    folds: List[Tuple[np.ndarray, np.ndarray]],
    k_list: List[int], out_dir: Path
) -> pd.DataFrame:
    """K-grid CV using only train data (prevent time series leakage)"""
    print(f"    Starting K-grid CV: K candidates={k_list} (train only)")
    rows = []
    rank = avg_importance["feature"].tolist()
    y_train = df_train[target_col].to_numpy()
    print(f"    - Train data: {len(df_train):,} rows")

    for K in k_list:
        print(f"      Processing K={K}...")
        kdir = out_dir / f"K{K}"
        kdir.mkdir(parents=True, exist_ok=True)

        topk = rank[:K]
        sel_num = [c for c in topk if c in num_cols]
        sel_cat = [c for c in topk if c in cat_cols]
        print(f"        - Selected features: {len(sel_num)} numeric + {len(sel_cat)} categorical = {len(topk)} total")

        fold_metrics = []
        for i, (tr_idx, va_idx) in enumerate(folds, start=1):
            fdir = kdir / f"fold{i}"
            fdir.mkdir(parents=True, exist_ok=True)

            # Fold split within train data
            X_tr = lgb_preprocess_for_lgb(df_train.iloc[tr_idx], sel_num, sel_cat)
            y_tr = y_train[tr_idx]
            X_va = lgb_preprocess_for_lgb(df_train.iloc[va_idx], sel_num, sel_cat)
            y_va = y_train[va_idx]

            booster, pred_va, rmse = lgb_train_eval(X_tr, y_tr, X_va, y_va, sel_cat)
            save_pickle(booster, fdir / "lgb_model.pkl")
            pd.DataFrame({"y": y_va, "pred": pred_va}).to_csv(fdir / "pred_val.csv", index=False)

            # Calculate R²
            ss_res = np.sum((y_va - pred_va) ** 2)
            ss_tot = np.sum((y_va - np.mean(y_va)) ** 2)
            r2 = 1.0 - (ss_res / (ss_tot + 1e-9))
            
            fold_metrics.append({"fold": i, "RMSE_va": rmse, "R2_va": r2})

            del booster, X_tr, X_va, y_tr, y_va, pred_va
            gc.collect()

        dfm = pd.DataFrame(fold_metrics)
        dfm.to_csv(kdir / "fold_metrics.csv", index=False)
        rmse_mean = float(dfm["RMSE_va"].mean())
        rmse_std = float(dfm["RMSE_va"].std(ddof=0))
        r2_mean = float(dfm["R2_va"].mean())
        r2_std = float(dfm["R2_va"].std(ddof=0))
        row = {"K": K, "RMSE_va_mean": rmse_mean, "RMSE_va_std": rmse_std, "R2_va_mean": r2_mean, "R2_va_std": r2_std}
        rows.append(row)
        print(f"        - RMSE: {rmse_mean:.6f} ± {rmse_std:.6f}, R²: {r2_mean:.6f} ± {r2_std:.6f}")

        del fold_metrics, dfm, topk, sel_num, sel_cat
        gc.collect()

    res = pd.DataFrame(rows).sort_values("RMSE_va_mean").reset_index(drop=True)
    res.to_csv(out_dir / "kgrid_cv_results.csv", index=False)
    print(f"    K-grid CV completed: Best K={res.iloc[0]['K']}, RMSE={res.iloc[0]['RMSE_va_mean']:.6f}, R²={res.iloc[0]['R2_va_mean']:.6f}")
    del rows, rank, y_train
    gc.collect()
    return res

# Poisson Regression or Negative Binomial Regression Setup (Select and regress based on the distribution of the target variable)

In [ ]:
def winsorize(train: pd.DataFrame, valid: pd.DataFrame, cols: List[str],
              lo_q=WINSOR_LO_Q, hi_q=WINSOR_HI_Q):
    tr = train.copy(); va = valid.copy()
    bounds = {}
    for c in cols:
        tr[c] = tr[c].astype(float)
        va[c] = va[c].astype(float)
        lo = np.nanquantile(tr[c], lo_q); hi = np.nanquantile(tr[c], hi_q)
        tr[c] = np.clip(tr[c], lo, hi)
        va[c] = np.clip(va[c], lo, hi)
        bounds[c] = {"lo": float(lo), "hi": float(hi)}
    return tr, va, bounds

def asinh_scale(train: pd.DataFrame, valid: pd.DataFrame, cols: List[str], q=ASINH_Q):
    tr = train.copy(); va = valid.copy(); scalers = {}
    for c in cols:
        base = np.asarray(tr[c].values, float)
        cval = np.quantile(np.abs(base[~np.isnan(base)]), q) if base.size else 1.0
        if not np.isfinite(cval) or cval <= 0: cval = 1.0
        tr[c] = np.arcsinh(np.asarray(tr[c].values, float) / cval)
        va[c] = np.arcsinh(np.asarray(va[c].values, float) / cval)
        scalers[c] = float(cval)
    return tr, va, scalers

def improved_asinh_scale(train: pd.DataFrame, valid: pd.DataFrame, cols: List[str]):
    """Improved asinh scaling for NB (learned from train data)"""
    tr = train.copy(); va = valid.copy(); scalers = {}
    for c in cols:
        base = np.asarray(tr[c].values, float)
        
        # Learn statistics from train data
        mean_val = np.nanmean(base)
        std_val = np.nanstd(base)
        q95 = np.nanquantile(np.abs(base), 0.95)
        
        # Calculate more appropriate scaling parameters
        if std_val > 0:
            # Standardization + asinh transformation
            scale_factor = max(std_val, q95 / 2.0)  # Larger of std and 95% quantile
        else:
            scale_factor = max(q95, 1.0)
        
        # Apply asinh transformation
        tr[c] = np.arcsinh((base - mean_val) / scale_factor)
        va[c] = np.arcsinh((np.asarray(va[c].values, float) - mean_val) / scale_factor)
        
        scalers[c] = {
            "mean": float(mean_val),
            "std": float(std_val),
            "scale_factor": float(scale_factor),
            "q95": float(q95)
        }
    return tr, va, scalers

def prepare_int_cats_for_ohe(
    train: pd.DataFrame, valid: pd.DataFrame, cat_int_cols: List[str], max_levels: int = 30
) -> Tuple[pd.DataFrame, pd.DataFrame, Dict[str, List[Any]]]:
    tr = train.copy(); va = valid.copy(); kept: Dict[str, List[Any]] = {}
    for c in cat_int_cols:
        if c not in tr.columns:
            continue
        tr_s = tr[c].astype("Int64").astype("string").fillna("__MISSING__")
        va_s = va[c].astype("Int64").astype("string").fillna("__MISSING__")
        top = (tr_s.value_counts(dropna=False).sort_values(ascending=False).head(max_levels).index.tolist())
        kept[c] = top; keep = set(top)
        tr_s = tr_s.where(tr_s.isin(keep), "__OTHER__")
        va_s = va_s.where(va_s.isin(keep), "__OTHER__")
        cats = list(dict.fromkeys(top + ["__OTHER__", "__MISSING__"]))
        tr[c] = pd.Categorical(tr_s, categories=cats)
        va[c] = pd.Categorical(va_s, categories=cats)
    return tr, va, kept

def one_hot_align(train: pd.DataFrame, valid: pd.DataFrame, cat_cols: List[str], drop_first=True):
    tr = pd.get_dummies(train, columns=cat_cols, drop_first=drop_first)
    va = pd.get_dummies(valid, columns=cat_cols, drop_first=drop_first)
    for c in tr.columns:
        if c not in va.columns:
            va[c] = 0
    va = va[tr.columns]
    tr = tr.apply(pd.to_numeric, errors="coerce").fillna(0.0).astype(float)
    va = va.apply(pd.to_numeric, errors="coerce").fillna(0.0).astype(float)
    return tr, va

In [ ]:
def fit_count_glm(X: pd.DataFrame, y: np.ndarray, alpha: float = 0.1, max_iter: int = 10000, tol: float = 1e-8):
    """
    Poisson regression with Ridge (log link). Learner that returns non-negative, stable μ.
    alpha is the strength of L2 regularization (default 0.1 for more flexibility).
    """
    print(f"          - Starting fit_count_glm: X.shape={X.shape}, y.shape={y.shape}, alpha={alpha}, max_iter={max_iter}, tol={tol}")
    
    # More flexible parameter settings
    model = PoissonRegressor(
        alpha=alpha, 
        fit_intercept=True, 
        max_iter=max_iter, 
        tol=tol,
        warm_start=False,
        solver='lbfgs'  # More stable solver
    )
    
    try:
        model.fit(np.asarray(X, float), y.astype(float))
        print(f"          - fit_count_glm completed: converged={model.n_iter_}, loss={model.score(np.asarray(X, float), y.astype(float)):.6f}")
        
        # Output parameter information
        print(f"          - Training parameters: alpha={alpha}, max_iter={max_iter}, tol={tol}")
        print(f"          - Convergence status: converged in {model.n_iter_} iterations")
        
    except Exception as e:
        print(f"          - fit_count_glm error: {e}")
        # Retry with more relaxed parameters
        print(f"          - Retrying: training with more relaxed parameters")
        model = PoissonRegressor(alpha=alpha*10, fit_intercept=True, max_iter=max_iter//2, tol=tol*10)
        model.fit(np.asarray(X, float), y.astype(float))
        print(f"          - Retry completed: converged={model.n_iter_}")
    
    return model

def poisson_grid_search(X: pd.DataFrame, y: np.ndarray, out_dir: Path) -> Tuple[Any, Dict[str, Any], float]:
    """Grid search for Poisson parameters"""
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import make_scorer, mean_squared_error
    
    print(f"        - Starting Poisson grid search...")
    
    # Grid search in a small region
    param_grid = {
        'alpha': [0.1,0.5,1.0],      # Regularization strength
        'max_iter': [5000],      # Number of iterations
        'tol': [1e-6]             # Convergence condition
    }
    
    best_score = -np.inf
    best_params = None
    best_model = None
    results = []
    
    # Execute grid search
    total_combinations = len(param_grid['alpha']) * len(param_grid['max_iter']) * len(param_grid['tol'])
    print(f"        - Total number of combinations: {total_combinations}")
    
    for i, alpha in enumerate(param_grid['alpha']):
        for j, max_iter in enumerate(param_grid['max_iter']):
            for k, tol in enumerate(param_grid['tol']):
                params = {'alpha': alpha, 'max_iter': max_iter, 'tol': tol}
                print(f"        - Trial {i*4 + j*2 + k + 1}/{total_combinations}: alpha={alpha}, max_iter={max_iter}, tol={tol}")
                
                try:
                    # Model training
                    model = fit_count_glm(X, y, **params)
                    
                    # Evaluate with 3-fold CV (3-fold for small datasets)
                    cv_scores = cross_val_score(
                        model, X, y, 
                        cv=3, 
                        scoring=make_scorer(lambda y_true, y_pred: -mean_squared_error(y_true, y_pred)),
                        n_jobs=1
                    )
                    mean_score = np.mean(cv_scores)
                    
                    print(f"          - CV score: {mean_score:.6f} (±{np.std(cv_scores):.6f})")
                    
                    results.append({
                        'params': params,
                        'cv_score': mean_score,
                        'cv_std': np.std(cv_scores),
                        'n_iter': model.n_iter_
                    })
                    
                    # Update best score
                    if mean_score > best_score:
                        best_score = mean_score
                        best_params = params
                        best_model = model
                        print(f"          - Best score updated: {best_score:.6f}")
                        
                except Exception as e:
                    print(f"          - Error: {e}")
                    results.append({
                        'params': params,
                        'cv_score': -np.inf,
                        'cv_std': 0.0,
                        'n_iter': 0,
                        'error': str(e)
                    })
    
    # Save results
    grid_results = {
        'best_params': best_params,
        'best_score': float(best_score),
        'all_results': results,
        'param_grid': param_grid
    }
    
    # Verify directory exists and save file
    try:
        out_dir.mkdir(parents=True, exist_ok=True)
        (out_dir / "poisson_grid_search.json").write_text(
            json.dumps(grid_results, ensure_ascii=False, indent=2)
        )
        print(f"        - Grid search results saved: {out_dir / 'poisson_grid_search.json'}")
    except Exception as e:
        print(f"        - Warning: Failed to save grid search results: {e}")
        # Continue processing even if file save fails
    
    print(f"        - Grid search completed: Best parameters={best_params}, score={best_score:.6f}")
    
    return best_model, best_params, best_score

def glm_predict(model, X: pd.DataFrame) -> np.ndarray:
    """PoissonRegressor prediction (mean μ). Always non-negative."""
    return np.asarray(model.predict(np.asarray(X, float)), float)

def cap_mu(mu: np.ndarray, y_ref: np.ndarray, q: float = 0.999, mult: float = 2.0) -> np.ndarray:
    """
    Upper limit control for predictions. Clip to q quantile × mult of training data as upper limit.
    Physically prevents "unrealistic" outliers.
    """
    cap = np.quantile(y_ref.astype(float), q) * mult
    if not np.isfinite(cap) or cap <= 0:
        return np.clip(mu, 0.0, None)
    return np.clip(mu, 0.0, cap)

# --- (Reference) Original NB model function ---
def fit_nb(X: pd.DataFrame, y: np.ndarray):
    X_np = np.asarray(X, dtype=float)
    exog = sm.add_constant(X_np, has_constant="add")
    
    # More stable initialization
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        warnings.filterwarnings("ignore", category=HessianInversionWarning)
        try:
            # Initialize with simpler Poisson model
            pois = PoisDM(y, exog).fit(method="lbfgs", maxiter=100, disp=False)
            beta0 = pois.params
        except Exception:
            # More conservative initialization
            beta0 = np.zeros(exog.shape[1])
            beta0[0] = np.log(np.mean(y) + 1e-6)  # Initialize intercept with log of mean
    
    # More conservative initialization of dispersion parameter
    start_params = np.r_[beta0, 0.5]  # 0.1 → 0.5
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        warnings.filterwarnings("ignore", category=HessianInversionWarning)
        model = NegativeBinomial(y, exog)
        # More iterations and more stable optimization
        res = model.fit(method="lbfgs", start_params=start_params, maxiter=1000, disp=False)
    return model, res

def nb_predict_with_se(res, X: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    X_np = np.asarray(X, dtype=float)
    exog = sm.add_constant(X_np, has_constant="add")
    mu = np.asarray(res.predict(exog), float)
    try:
        cov = np.asarray(res.cov_params())
        var_eta = np.einsum('ij,jk,ik->i', exog, cov, exog)
        var_eta = np.clip(var_eta, 0.0, None)
        se_mu = mu * np.sqrt(var_eta)  # Delta method
    except Exception:
        se_mu = np.full_like(mu, np.nan, dtype=float)
    return mu, se_mu

# Perform Poisson or NB Regression for Final Model Training and Introduce Results as Features

In [ ]:
def build_nb_features_for_val_from_train(
    df_train: pd.DataFrame, df_val: pd.DataFrame, target_col: str,
    selected_num_cols: List[str], selected_cat_cols: List[str]
) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """Generate val features using NB model trained on train data (prevent time series leakage)"""
    print(f"    Starting val NB feature generation (using train-trained model)")
    
    use_feats = selected_num_cols + selected_cat_cols
    sel_num = [c for c in use_feats if c in selected_num_cols]
    sel_cat = [c for c in use_feats if c in selected_cat_cols]
    print(f"    - Features used: {len(sel_num)} numeric + {len(sel_cat)} categorical = {len(use_feats)} total")

    # Preprocess train data
    tr_df = df_train[sel_num + sel_cat].copy()
    va_df = df_val[sel_num + sel_cat].copy()
    
    print(f"    - Train data: {len(tr_df):,} rows")
    print(f"    - Val data: {len(va_df):,} rows")

    # Preprocess numeric features (determine bounds from train statistics)
    print(f"    - Preprocessing numeric features...")
    for c in sel_num:
        tr_df[c] = tr_df[c].astype(float).fillna(0.0)
        va_df[c] = va_df[c].astype(float).fillna(0.0)
    
    # Preprocessing for NB: only mild winsorization (asinh scaling removed)
    Tr_cont, Va_cont, bounds = winsorize(tr_df[sel_num], va_df[sel_num], sel_num, 5.0/100, 95.0/100)
    # Remove asinh scaling and use as is
    improved_scalers = {}
    tr_df[sel_num] = Tr_cont; va_df[sel_num] = Va_cont

    # OHE transformation for categorical features (fixed to train levels)
    print(f"    - OHE transforming categorical features...")
    tr_cat_ready, va_cat_ready, _ = prepare_int_cats_for_ohe(tr_df, va_df, sel_cat, max_levels=30)
    X_tr, X_va = one_hot_align(tr_cat_ready, va_cat_ready, sel_cat, drop_first=True)
    print(f"    - Number of features after OHE: {X_tr.shape[1]}")

    # Train NB on entire train data
    y_tr = df_train[target_col].to_numpy()
    print(f"    - Training Poisson(Ridge)...")
    model = fit_count_glm(X_tr, y_tr, alpha=1.0)

    # Predict for val (mu only)
    print(f"    - Predicting for val...")
    mu_va = glm_predict(model, X_va)
    mu_va = cap_mu(mu_va, y_tr, q=0.999, mult=2.0)
    
    # Calculate R² for val NB predictions
    y_val = df_val[target_col].to_numpy()
    ss_res = np.sum((y_val - mu_va) ** 2)
    ss_tot = np.sum((y_val - np.mean(y_val)) ** 2)
    r2 = 1.0 - (ss_res / (ss_tot + 1e-9))
    
    print(f"    - Val NB prediction completed: mean prediction={np.mean(mu_va):.3f}, R²={r2:.6f}")
    print(f"    - Actual value statistics: mean={np.mean(y_val):.3f}, min={np.min(y_val):.3f}, max={np.max(y_val):.3f}")
    print(f"    - Prediction statistics: mean={np.mean(mu_va):.3f}, min={np.min(mu_va):.3f}, max={np.max(mu_va):.3f}")

    # Create val features (mu only)
    nb_feat_val = pd.DataFrame({"nb_mu": mu_va})
    
    # Save preprocessing information
    preproc_info = {
        "winsor_bounds": bounds,
        "improved_scalers": improved_scalers,
        "sel_num": sel_num,
        "sel_cat": sel_cat,
        "ohe_columns": list(X_tr.columns)
    }
    
    return nb_feat_val, preproc_info

def build_nb_oof_features(
    df_train: pd.DataFrame, target_col: str,
    top_rank: List[str], num_cols: List[str], cat_cols: List[str],
    folds: List[Tuple[np.ndarray, np.ndarray]],
    out_dir: Path, nb_top: int = NB_TOP
) -> pd.DataFrame:
    """Generate NB OOF features using only train data (prevent time series leakage)"""
    print(f"    Starting Negative Binomial OOF feature generation (train only, top {nb_top} features)")
    os.makedirs(out_dir, exist_ok=True)
    use_feats = top_rank[:nb_top]
    sel_num = [c for c in use_feats if c in num_cols]
    sel_cat = [c for c in use_feats if c in cat_cols]
    print(f"    - Features used: {len(sel_num)} numeric + {len(sel_cat)} categorical = {len(use_feats)} total")
    print(f"    - Train data: {len(df_train):,} rows")

    oof_mu = np.full(len(df_train), np.nan, dtype=float)
    y_train = df_train[target_col].to_numpy()

    for i, (tr_idx, va_idx) in enumerate(folds, start=1):
        print(f"      Processing NB Fold {i}/{len(folds)}...")
        fdir = out_dir / f"fold{i}"
        fdir.mkdir(parents=True, exist_ok=True)

        # Fold split within train data
        tr_df = df_train.iloc[tr_idx][sel_num + sel_cat].copy()
        va_df = df_train.iloc[va_idx][sel_num + sel_cat].copy()
        print(f"        - Training data: {len(tr_df):,} rows")
        print(f"        - Validation data: {len(va_df):,} rows")

        print(f"        - Preprocessing numeric features...")
        for c in sel_num:
            tr_df[c] = tr_df[c].astype(float).fillna(0.0)
            va_df[c] = va_df[c].astype(float).fillna(0.0)
        # Preprocessing for NB: only mild winsorization (asinh scaling removed)
        Tr_cont, Va_cont, bounds = winsorize(tr_df[sel_num], va_df[sel_num], sel_num, 5.0/100, 95.0/100)
        # Remove asinh scaling and use as is
        improved_scalers = {}
        tr_df[sel_num] = Tr_cont; va_df[sel_num] = Va_cont

        print(f"        - OHE transforming categorical features...")
        tr_cat_ready, va_cat_ready, _ = prepare_int_cats_for_ohe(tr_df, va_df, sel_cat, max_levels=30)
        X_tr, X_va = one_hot_align(tr_cat_ready, va_cat_ready, sel_cat, drop_first=True)
        print(f"        - Number of features after OHE: {X_tr.shape[1]}")

        y_tr = y_train[tr_idx]; y_va = y_train[va_idx]
        print(f"        - Training Poisson(Ridge)... (train: {len(y_tr):,} rows, val: {len(y_va):,} rows)")
        print(f"        - y_tr statistics: mean={np.mean(y_tr):.3f}, min={np.min(y_tr):.3f}, max={np.max(y_tr):.3f}")
        model = fit_count_glm(X_tr, y_tr, alpha=1.0)
        print(f"        - Poisson training completed")
        save_pickle(model, fdir / "nb_model.pkl")  # Keep filename for compatibility
        (fdir / "numeric_preproc.json").write_text(
            json.dumps({"winsor_bounds": bounds, "improved_scalers": improved_scalers,
                        "sel_num": sel_num, "sel_cat": sel_cat}, ensure_ascii=False, indent=2)
        )

        print(f"        - Predicting...")
        mu_va = glm_predict(model, X_va)
        mu_va = cap_mu(mu_va, y_tr, q=0.999, mult=2.0)
        se_va = np.full_like(mu_va, np.nan, dtype=float)  # Keep column for compatibility

        oof_mu[va_idx] = mu_va

        # Calculate R² for NB predictions
        ss_res = np.sum((y_va - mu_va) ** 2)
        ss_tot = np.sum((y_va - np.mean(y_va)) ** 2)
        r2 = 1.0 - (ss_res / (ss_tot + 1e-9))

        pd.DataFrame({"y": y_va, "nb_mu": mu_va, "nb_se": se_va}).to_csv(
            fdir / "nb_val_preds.csv", index=False
        )
        print(f"        - NB prediction completed: mean prediction={np.mean(mu_va):.3f}, R²={r2:.6f}")
        print(f"        - Actual value statistics: mean={np.mean(y_va):.3f}, min={np.min(y_va):.3f}, max={np.max(y_va):.3f}")
        print(f"        - Prediction statistics: mean={np.mean(mu_va):.3f}, min={np.min(mu_va):.3f}, max={np.max(mu_va):.3f}")

        print(f"        - Fold {i} processing completed")
        del tr_df, va_df, Tr_cont, Va_cont, X_tr, X_va, y_tr, y_va, model, mu_va, se_va
        gc.collect()

    print(f"    Aggregating NB OOF features...")
    nb_feat = pd.DataFrame({"nb_mu": oof_mu})
    nb_feat.to_csv(out_dir / "nb_oof_features.csv", index=False)
    print(f"    - NB feature generation completed: {len(nb_feat.columns)} features")

    del oof_mu, y_train
    gc.collect()
    return nb_feat

def build_poisson_features_full_data(
    df_train: pd.DataFrame, df_val: pd.DataFrame, target_col: str,
    selected_num_cols: List[str], selected_cat_cols: List[str], out_dir: Path
) -> Tuple[pd.DataFrame, pd.DataFrame, Dict[str, Any]]:
    """Train Poisson using only train data → apply inference to both train/val"""
    print(f"    - Starting Poisson training with train data only (top {len(selected_num_cols + selected_cat_cols)} features)")
    
    # Prepare features using only train data
    tr_df = df_train[selected_num_cols + selected_cat_cols].copy()
    va_df = df_val[selected_num_cols + selected_cat_cols].copy()
    print(f"    - Features used: {len(selected_num_cols)} numeric + {len(selected_cat_cols)} categorical")
    print(f"    - Train data: {len(tr_df):,} rows, val data: {len(va_df):,} rows")
    
    # Preprocess numeric features (determine bounds from train statistics)
    print(f"    - Preprocessing numeric features...")
    for c in selected_num_cols:
        tr_df[c] = tr_df[c].astype(float).fillna(0.0)
        va_df[c] = va_df[c].astype(float).fillna(0.0)
    
    # Mild winsorization (determine bounds from train statistics)
    Tr_cont, Va_cont, bounds = winsorize(tr_df[selected_num_cols], va_df[selected_num_cols], selected_num_cols, 5.0/100, 95.0/100)
    
    # Scale numeric features (standardize using train statistics)
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    Tr_cont_scaled = pd.DataFrame(
        scaler.fit_transform(Tr_cont), 
        columns=Tr_cont.columns, 
        index=Tr_cont.index
    )
    Va_cont_scaled = pd.DataFrame(
        scaler.transform(Va_cont), 
        columns=Va_cont.columns, 
        index=Va_cont.index
    )
    
    tr_df[selected_num_cols] = Tr_cont_scaled
    va_df[selected_num_cols] = Va_cont_scaled
    
    # Check statistics after scaling
    print(f"    - Statistics after scaling: train mean={np.mean(Tr_cont_scaled.values):.3f}, val mean={np.mean(Va_cont_scaled.values):.3f}")
    print(f"    - Range after scaling: train[{np.min(Tr_cont_scaled.values):.3f}, {np.max(Tr_cont_scaled.values):.3f}], val[{np.min(Va_cont_scaled.values):.3f}, {np.max(Va_cont_scaled.values):.3f}]")
    
    # OHE transformation for categorical features (fixed to train levels)
    print(f"    - OHE transforming categorical features...")
    tr_cat_ready, va_cat_ready, _ = prepare_int_cats_for_ohe(tr_df, va_df, selected_cat_cols, max_levels=30)
    X_tr, X_va = one_hot_align(tr_cat_ready, va_cat_ready, selected_cat_cols, drop_first=True)
    print(f"    - Number of features after OHE: {X_tr.shape[1]}")
    
    # Train Poisson using only train data
    y_tr = df_train[target_col].to_numpy()
    print(f"    - Training Poisson(Ridge)... (train only: {len(y_tr):,} rows)")
    print(f"    - y statistics: mean={np.mean(y_tr):.3f}, min={np.min(y_tr):.3f}, max={np.max(y_tr):.3f}")
    
    # Parameter grid search
    model, best_params, best_score = poisson_grid_search(X_tr, y_tr, out_dir)
    print(f"    - Poisson training completed (best score: {best_score:.6f})")
    
    # Save model and parameters
    save_pickle(model, out_dir / "poisson_model.pkl")
    
    # Save parameter information
    model_info = {
        "poisson_params": best_params,
        "best_score": float(best_score),
        "n_iter": int(model.n_iter_),
        "n_features": int(X_tr.shape[1]),
        "train_size": int(len(y_tr)),
        "converged": bool(model.n_iter_ < best_params["max_iter"])
    }
    try:
        (out_dir / "poisson_model_info.json").write_text(
            json.dumps(model_info, ensure_ascii=False, indent=2)
        )
        print(f"    - Model information saved: {out_dir / 'poisson_model_info.json'}")
    except Exception as e:
        print(f"    - Warning: Failed to save model information: {e}")
    
    # Apply inference to both train/val
    print(f"    - Predicting for train... ({len(X_tr):,} rows)")
    mu_train = glm_predict(model, X_tr)
    mu_train = cap_mu(mu_train, y_tr, q=0.999, mult=2.0)
    
    print(f"    - Predicting for val... ({len(X_va):,} rows)")
    mu_val = glm_predict(model, X_va)
    mu_val = cap_mu(mu_val, y_tr, q=0.999, mult=2.0)  # Clip using train statistics
    
    # Create features
    nb_feats_train = pd.DataFrame({"nb_mu": mu_train})
    nb_feats_val = pd.DataFrame({"nb_mu": mu_val})
    
    # Calculate R²
    y_val = df_val[target_col].to_numpy()
    
    ss_res_train = np.sum((y_tr - mu_train) ** 2)
    ss_tot_train = np.sum((y_tr - np.mean(y_tr)) ** 2)
    r2_train = 1.0 - (ss_res_train / (ss_tot_train + 1e-9))
    
    ss_res_val = np.sum((y_val - mu_val) ** 2)
    ss_tot_val = np.sum((y_val - np.mean(y_val)) ** 2)
    r2_val = 1.0 - (ss_res_val / (ss_tot_val + 1e-9))
    
    print(f"    - train R²: {r2_train:.6f}, val R²: {r2_val:.6f}")
    print(f"    - train prediction statistics: mean={np.mean(mu_train):.3f}, min={np.min(mu_train):.3f}, max={np.max(mu_train):.3f}")
    print(f"    - val prediction statistics: mean={np.mean(mu_val):.3f}, min={np.min(mu_val):.3f}, max={np.max(mu_val):.3f}")
    
    # Check prediction diversity
    train_unique = len(np.unique(mu_train))
    val_unique = len(np.unique(mu_val))
    print(f"    - Prediction diversity: train={train_unique:,} types, val={val_unique:,} types")
    
    if train_unique < 10 or val_unique < 10:
        print(f"    Warning: Prediction diversity may be low")
        print(f"    - Consider adjusting parameters: alpha={best_params['alpha']}")
    
    # Preprocessing information
    preproc_info = {
        "winsor_bounds": bounds,
        "scaler_mean": scaler.mean_.tolist(),
        "scaler_scale": scaler.scale_.tolist(),
        "selected_num": selected_num_cols,
        "selected_cat": selected_cat_cols,
        "ohe_columns": list(X_tr.columns),
        "poisson_params": best_params,
        "model_info": model_info,
        "r2_train": float(r2_train),
        "r2_val": float(r2_val)
    }
    
    return nb_feats_train, nb_feats_val, preproc_info

# Final LightGBM Training (Step 3)
- Step 3 : Train LightGBM using K+1 features (K features selected in Step 2 + output values from Poisson or NB regression)
           (No CV, split all period data into 8:2 (train/val) for training)

In [ ]:
def lgb_with_nbfeatures_fulltrain(
    df_train: pd.DataFrame, df_val: pd.DataFrame, target_col: str,
    avg_importance: pd.DataFrame,
    nb_feats_train: pd.DataFrame, nb_feats_val: pd.DataFrame,
    num_cols: List[str], cat_cols: List[str],
    best_K: int, out_dir: Path
) -> Dict[str, Any]:
    """Final LightGBM with train training → val prediction (prevent time series leakage)"""
    print(f"    Starting final LightGBM training (K={best_K})")
    rank = avg_importance["feature"].tolist()
    base_feats = rank[:best_K]
    base_num = [c for c in base_feats if c in num_cols]
    base_cat = [c for c in base_feats if c in cat_cols]
    print(f"    - Base features: {len(base_num)} numeric + {len(base_cat)} categorical")

    final_num = base_num + ["nb_mu"]
    final_cat = base_cat
    print(f"    - Final features: {len(final_num)} numeric + {len(final_cat)} categorical = {len(final_num + final_cat)} total")

    kdir = out_dir / f"final_full_with_nb_K{best_K}"
    kdir.mkdir(parents=True, exist_ok=True)

    # Training on train data
    print(f"    - Preparing train data...")
    df_train_aug = df_train[base_num + base_cat].copy().reset_index(drop=True)
    df_train_aug = pd.concat([df_train_aug, nb_feats_train.reset_index(drop=True)], axis=1)
    print(f"    - Number of features after train concatenation: {len(df_train_aug.columns)}")

    # Prepare val data
    print(f"    - Preparing val data...")
    df_val_aug = df_val[base_num + base_cat].copy().reset_index(drop=True)
    df_val_aug = pd.concat([df_val_aug, nb_feats_val.reset_index(drop=True)], axis=1)

    print(f"    - Preprocessing train...")
    X_train = lgb_preprocess_for_lgb(df_train_aug, final_num, final_cat)
    y_train = df_train[target_col].to_numpy()
    print(f"    - Train training data: {len(X_train):,} rows × {X_train.shape[1]} features")

    print(f"    - Preprocessing val...")
    X_val = lgb_preprocess_for_lgb(df_val_aug, final_num, final_cat)
    y_val = df_val[target_col].to_numpy()
    print(f"    - Val prediction data: {len(X_val):,} rows × {X_val.shape[1]} features")

    print(f"    - Training on train...")
    booster_full, _, _ = lgb_train_eval(X_train, y_train, X_val, y_val, final_cat)
    save_pickle(booster_full, kdir / "lgb_model_full.pkl")

    print(f"    - Predicting on val...")
    pred_val = booster_full.predict(X_val, num_iteration=booster_full.best_iteration)
    
    # Save prediction results
    pd.DataFrame({"y_val": y_val, "yhat_val": pred_val}).to_csv(
        kdir / "val_predictions.csv", index=False
    )

    # Calculate R² for final training
    ss_res = np.sum((y_val - pred_val) ** 2)
    ss_tot = np.sum((y_val - np.mean(y_val)) ** 2)
    val_r2 = 1.0 - (ss_res / (ss_tot + 1e-9))
    
    summary = {
        "final_features": {"numeric": final_num, "categorical": final_cat},
        "n_train": int(len(X_train)),
        "n_val": int(len(X_val)),
        "best_K": int(best_K),
        "train_only": True,
        "val_rmse": float(np.sqrt(np.mean((y_val - pred_val) ** 2))),
        "val_r2": val_r2
    }
    (kdir / "summary.json").write_text(json.dumps(summary, ensure_ascii=False, indent=2))
    print(f"    - Final model training completed: train={len(X_train):,}, val={len(X_val):,}, val_RMSE={summary['val_rmse']:.6f}, val_R²={summary['val_r2']:.6f}")

    del df_train_aug, df_val_aug, X_train, X_val, y_train, y_val, pred_val, booster_full
    gc.collect()

    return {"summary": summary, "model_path": str((kdir / "lgb_model_full.pkl").resolve())}

# Training Pipeline Setup (Configure flow of functions above) and Evaluation Function Setup

In [ ]:
def load_task_df_and_splitinfo(task: str) -> Tuple[pd.DataFrame, str, int, int, pd.Timestamp]:
    """Return data loading and time series split information"""
    df, target = load_src_df(task)
    cutoff_date, tr_idx, va_idx = time_split_by_date(df, train_ratio=0.8, date_col=DATE_COL)
    n_train, n_val = len(tr_idx), len(va_idx)
    return df, target, n_train, n_val, cutoff_date

def get_train_val_split(df_all: pd.DataFrame, cutoff_date: pd.Timestamp) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Split train/val from all data (prevent time series leakage)"""
    train_mask = df_all[DATE_COL] <= cutoff_date
    val_mask = df_all[DATE_COL] > cutoff_date
    
    df_train = df_all[train_mask].copy().reset_index(drop=True)
    df_val = df_all[val_mask].copy().reset_index(drop=True)
    
    print(f"        - train/val split: train={len(df_train):,} rows, val={len(df_val):,} rows")
    return df_train, df_val

def run_pipeline_for_task(task: str):
    print(f"  Creating output directory for task '{task}'...")
    out_dir = ROOT_OUT_DIR / task
    out_dir.mkdir(parents=True, exist_ok=True)

    print(f" Loading data & time series splitting...")
    df_all, target_col, n_train, n_val, cutoff_date = load_task_df_and_splitinfo(task)
    print(f"    - Total data count: {len(df_all):,}")
    print(f"    - Target variable: {target_col}")
    print(f"    - Training data count: {n_train:,} (80%)")
    print(f"    - Validation data count: {n_val:,} (20%)")
    print(f"    - Split cutoff date: {cutoff_date}")

    (out_dir / "basic_info.json").write_text(json.dumps({
        "task": task, "target": target_col,
        "n_all": int(len(df_all)), "n_train_time80": n_train, "n_val_time20": n_val,
        "cutoff_date": str(cutoff_date)
    }, ensure_ascii=False, indent=2))

    print(f" Selecting features...")
    num_all, cat_all = pick_feature_cols(df_all, target_col)
    print(f"    - Number of numeric features: {len(num_all)}")
    print(f"    - Number of categorical features: {len(cat_all)}")

    # Split train/val
    df_train, df_val = get_train_val_split(df_all, cutoff_date)
    
    print(f" Setting up 3-fold CV...")
    # 3-fold CV within train data (based on sold_date)
    folds_train = make_3fold_80_20_indices(df_train, date_col=DATE_COL)
    (out_dir / "folds.json").write_text(json.dumps(
        [{"fold": i+1, "n_train": int(len(tr)), "n_val": int(len(va))} for i, (tr, va) in enumerate(folds_train)],
        ensure_ascii=False, indent=2)
    )
    print(f"    - Fold setup completed: {len(folds_train)} folds")

    print(f"  Step 1/4: Calculating LightGBM feature importance...")
    imp_dir = out_dir / "importance_cv"
    avg_imp = lgb_fold_importances_raw(df_train, target_col, num_all, cat_all, folds_train, imp_dir)
    print(f"    - Importance calculation completed: {len(avg_imp)} features")

    print(f"  Step 2/4: Running K-grid CV...")
    kcv_dir = out_dir / "kgrid_lgb_cv"
    kres = lgb_kgrid_cv_raw(df_train, target_col, num_all, cat_all, avg_imp, folds_train, K_CANDIDATES, kcv_dir)
    best_row = kres.iloc[0].to_dict()
    best_K = int(best_row["K"])
    (out_dir / "best_k.json").write_text(json.dumps(best_row, ensure_ascii=False, indent=2))
    print(f"    - Best K: {best_K}")
    print(f"    - Best RMSE: {best_row['RMSE_va_mean']:.6f}")

    print(f"  Step 3/4: Generating Poisson features...")
    nb_dir = out_dir / "nb_oof"
    
    # Train Poisson model with all data combined
    selected_features = avg_imp["feature"].tolist()[:NB_TOP]
    selected_num = [c for c in selected_features if c in num_all]
    selected_cat = [c for c in selected_features if c in cat_all]
    
    nb_feats_train, nb_feats_val, nb_preproc_info = build_poisson_features_full_data(
        df_train, df_val, target_col, selected_num, selected_cat, nb_dir
    )
    print(f"    - train Poisson feature generation completed: {len(nb_feats_train.columns)} features")
    print(f"    - val Poisson feature generation completed: {len(nb_feats_val.columns)} features")
    
    # Save preprocessing information
    (nb_dir / "preproc_info.json").write_text(
        json.dumps(nb_preproc_info, ensure_ascii=False, indent=2)
    )

    print(f"  Step 4/4: Final LightGBM training...")
    final = lgb_with_nbfeatures_fulltrain(df_train, df_val, target_col, avg_imp, nb_feats_train, nb_feats_val, num_all, cat_all, best_K, out_dir)
    (out_dir / "final_summary.json").write_text(json.dumps(final, ensure_ascii=False, indent=2))
    print(f"    - Final model training completed")

    (out_dir / "time_split.json").write_text(json.dumps({
        "n_train": n_train, "n_val": n_val, "cutoff_date": str(cutoff_date)
    }, ensure_ascii=False, indent=2))

    print(f"  Task '{task}' completed: Best K={best_K}, RMSE_va_mean={best_row['RMSE_va_mean']:.6f}")
    print(f"  Output location: {out_dir.resolve()}")

    # Free memory after each training step
    print(f"  Freeing memory...")
    to_delete = [df_all, df_train, df_val, folds_train, avg_imp, kres, best_row, nb_feats_train, nb_feats_val, final, num_all, cat_all]
    for obj in to_delete:
        try: del obj
        except Exception: pass
    gc.collect()
    print(f"  Memory freed")

In [ ]:
def load_final_feature_spec(task_dir: Path) -> Tuple[List[str], List[str], str]:
    cand = sorted(task_dir.glob("final_full_with_nb_K*/summary.json"))
    if not cand:
        raise FileNotFoundError(f"No final summary.json under {task_dir}")
    summ_path = cand[0]
    with open(summ_path, "r") as f:
        summ = json.load(f)
    feats = summ["final_features"]
    final_num = feats["numeric"]
    final_cat = feats["categorical"]
    model_path = str(summ_path.parent.joinpath("lgb_model_full.pkl"))
    return final_num, final_cat, model_path

def load_time_split_counts(task: str) -> Tuple[int, int, pd.Timestamp]:
    """Load time series split information from training time"""
    task_dir = ROOT_OUT_DIR / task
    sp = task_dir / "time_split.json"
    if sp.exists():
        js = json.loads(sp.read_text())
        return int(js["n_train"]), int(js["n_val"]), pd.to_datetime(js["cutoff_date"])
    df, _ = load_src_df(task)
    cutoff, tr, va = time_split_by_date(df, 0.8, DATE_COL)
    return len(tr), len(va), cutoff

def build_X_for_inference(df_base: pd.DataFrame, nb_val: pd.DataFrame,
                          final_num: List[str], final_cat: List[str]) -> pd.DataFrame:
    base_num = [c for c in final_num if c not in ("nb_mu")]
    X_num = df_base.reindex(columns=base_num, fill_value=0.0).copy()
    nb_cols = ["nb_mu"]
    for c in nb_cols:
        if c not in nb_val.columns:
            raise KeyError(f"nb features missing: {c}")
    X_nb = nb_val[nb_cols].copy()
    X_cat = df_base.reindex(columns=final_cat, fill_value=-1).copy()
    X_all = pd.concat([X_num, X_nb, X_cat], axis=1)
    X_all = lgb_preprocess_for_lgb(X_all, final_num, final_cat)
    assert list(X_all.columns) == (final_num + final_cat), "Column order mismatch"
    return X_all

def load_nb_features_for_val(task_dir: Path) -> pd.DataFrame:
    """Load val NB features"""
    # Load val NB features directly
    val_pred_path = task_dir / "final_full_with_nb_K*" / "val_predictions.csv"
    val_pred_candidates = list(task_dir.glob("final_full_with_nb_K*/val_predictions.csv"))
    
    if not val_pred_candidates:
        raise FileNotFoundError(f"No val predictions found in {task_dir}")
    
    val_pred_path = val_pred_candidates[0]
    val_pred_df = pd.read_csv(val_pred_path)
    
    # Reconstruct NB features (from val predictions)
    nb_val = pd.DataFrame({"nb_mu": val_pred_df["yhat_val"]})
    return nb_val

def evaluate_single_task_val(task: str) -> Tuple[np.ndarray, np.ndarray]:
    """Evaluate single task on validation data (prevent time series leakage)"""
    # Get val data with same split as training time
    df_all, target_col = load_src_df(task)
    _, _, cutoff_date = load_time_split_counts(task)
    df_all = df_all.sort_values(DATE_COL).reset_index(drop=True)
    val_mask = df_all[DATE_COL] > cutoff_date
    y_val = df_all.loc[val_mask, target_col].to_numpy()

    task_dir = ROOT_OUT_DIR / task
    final_num, final_cat, model_path = load_final_feature_spec(task_dir)

    # Load val NB features
    nb_val = load_nb_features_for_val(task_dir)

    # Prepare base features
    use_base_cols = [c for c in (final_num + final_cat) if c not in ("nb_mu")]
    df_val_base = df_all.loc[val_mask, use_base_cols].reindex(columns=use_base_cols, fill_value=0).reset_index(drop=True)

    # Build inference features
    X_val = build_X_for_inference(df_val_base, nb_val, final_num, final_cat)

    # Predict with trained model
    booster: lgb.Booster = pickle.load(open(model_path, "rb"))
    yhat = booster.predict(X_val, num_iteration=booster.best_iteration)

    # Save evaluation results
    (task_dir / "eval").mkdir(parents=True, exist_ok=True)
    pd.DataFrame({"y_val": y_val, "yhat": yhat}).to_csv(task_dir / "eval" / "val_predictions.csv", index=False)

    return y_val, yhat

def evaluate_sum_task_fixed():
    y_val, yhat = evaluate_single_task_val("sum")
    return y_val, yhat, None

def evaluate_horizon_sum_vs_sumval_fixed():
    """Compare sum of h1-h26 with sum validation data"""
    df_sum, _ = load_src_df("sum")
    _, _, cutoff_date = load_time_split_counts("sum")
    df_sum = df_sum.sort_values(DATE_COL).reset_index(drop=True)
    y_true_sum = df_sum.loc[df_sum[DATE_COL] > cutoff_date, SUM_TARGET].to_numpy()

    preds = []
    for N in range(1, 27):
        task = f"h{str(N).zfill(2)}"
        _, yhat_h = evaluate_single_task_val(task)
        preds.append(yhat_h)

    yhat_sum = np.sum(np.vstack(preds), axis=0)
    return y_true_sum, yhat_sum

def compute_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    """Calculate evaluation metrics"""
    y_true = np.asarray(y_true, float)
    y_pred = np.asarray(y_pred, float)
    eps = 1e-9
    rmse = float(np.sqrt(np.mean((y_pred - y_true) ** 2)))
    mae = float(np.mean(np.abs(y_pred - y_true)))
    smape = float(np.mean(2.0 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true) + eps)))
    ss_res = float(np.sum((y_true - y_pred) ** 2))
    ss_tot = float(np.sum((y_true - np.mean(y_true)) ** 2) + eps)
    r2 = float(1.0 - ss_res / ss_tot)
    bias = float(np.mean(y_pred - y_true))
    return {
        "n_val": int(y_true.size),
        "rmse": rmse,
        "mae": mae,
        "smape": smape,
        "r2": r2,
        "bias_mean": bias,
        "sum_true": float(np.sum(y_true)),
        "sum_pred": float(np.sum(y_pred)),
        "sum_diff": float(np.sum(y_pred) - np.sum(y_true)),
    }

def run_all_evaluations_and_write():
    print("  🔍 Starting evaluation of all tasks")
    rows = []

    tasks = ["sum"] + [f"h{str(n).zfill(2)}" for n in range(1, 27)]
    print(f"  Tasks to evaluate: {len(tasks)} tasks")
    
    for i, task in enumerate(tasks, 1):
        print(f"    [{i:2d}/27] Evaluating task '{task}'...")
        task_dir = ROOT_OUT_DIR / task
        try:
            if not (task_dir.exists() and list(task_dir.glob("final_full_with_nb_K*/summary.json"))):
                print(f"      Skipped: Final model not found")
                continue
            y_val, yhat = evaluate_single_task_val(task)
            m = compute_metrics(y_val, yhat)
            m["task"] = task
            rows.append(m)
            print(f"      Completed: RMSE={m['rmse']:.6f}  MAE={m['mae']:.6f}  sMAPE={m['smape']:.6f}")
        except Exception as e:
            print(f"      Error: {e}")

    print(f"  Running aggregate evaluation...")
    try:
        y_true_sum, yhat_sum = evaluate_horizon_sum_vs_sumval_fixed()
        m_sum = compute_metrics(y_true_sum, yhat_sum)
        m_sum["task"] = "h_sum_vs_sum_val"
        rows.append(m_sum)
        print(f"  Aggregate evaluation completed: RMSE={m_sum['rmse']:.6f}  MAE={m_sum['mae']:.6f}  sMAPE={m_sum['smape']:.6f}")
    except Exception as e:
        print(f"  Aggregate evaluation error: {e}")

    if rows:
        print(f"  Aggregating evaluation results...")
        df_metrics = pd.DataFrame(rows)[[
            "task","n_val","rmse","mae","smape","r2","bias_mean","sum_true","sum_pred","sum_diff"
        ]].sort_values("task").reset_index(drop=True)
        df_metrics.to_csv(EVAL_CSV_PATH, index=False)
        with open(EVAL_JSON_PATH, "w") as f:
            json.dump({"metrics": rows}, f, ensure_ascii=False, indent=2)
        print(f"  Aggregated CSV: {EVAL_CSV_PATH.resolve()}")
        print(f"  JSON:     {EVAL_JSON_PATH.resolve()}")
        print(f"  Evaluation completed: {len(rows)} tasks")
    else:
        print("  No output targets.")

# Pipeline Execution

In [ ]:
def main():
    print("=" * 80)
    print("Time Series Leak Prevention Machine Learning Pipeline Started")
    print("=" * 80)
    print(f"Output directory: {ROOT_OUT_DIR}")
    print(f"Number of tasks to process: 27 (sum + h1-h26)")
    print(f"Target variable base: {TARGET_BASE}")
    print(f"LightGBM settings: {LGB_NUM_BOOST_ROUND} training rounds, early_stopping={LGB_EARLY_STOP_ROUNDS}")
    print(f"K candidates: {K_CANDIDATES}")
    print(f"NB top features: {NB_TOP}")
    print("Time series leak prevention: strict train/val split, no date crossing")
    print("=" * 80)
    
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    warnings.filterwarnings("ignore", category=HessianInversionWarning)

    tasks = ["sum"] + [f"h{str(n).zfill(2)}" for n in range(1, 27)]
    print(f"Task list to process: {tasks}")
    print("=" * 80)
    
    for i, task in enumerate(tasks, 1):
        print(f"\n[{i:2d}/27] Starting task '{task}'...")
        try:
            run_pipeline_for_task(task)
            print(f"[{i:2d}/27] Task '{task}' completed")
        except Exception as e:
            print(f"[{i:2d}/27] Task '{task}' error: {e}")
        finally:
            gc.collect()

    print("\n" + "=" * 80)
    print("Training phase completed")
    print("=" * 80)
    print(f"Output root: {ROOT_OUT_DIR.resolve()}")

    print("\n" + "=" * 80)
    print("Validation phase started")
    print("=" * 80)
    run_all_evaluations_and_write()
    print("\n" + "=" * 80)
    print("Validation of all tasks and aggregate completed")
    print("=" * 80)
    print(f"Validation CSV: {EVAL_CSV_PATH.resolve()}")
# --- Set warnings to be hidden ---
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=HessianInversionWarning)

# --- Execute pipeline ---
main()